# Clouds

##### A GWDST project to analyze and visualize modeled cloud features, particularly variability and extremes.

### Input and data

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import xarray as xr
import xarray.ufuncs as xu
import matplotlib.pyplot as plt
import seaborn as sns
#import cartopy.crs as ccrs

from dask.distributed import Client

sns.set(context='talk', palette='colorblind', style='ticks')

In [2]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:43528 Dashboard: http://127.0.0.1:8787,Cluster Workers: 72 Cores: 72 Memory: 81.05 GB


In [3]:
ds = xr.open_mfdataset('/glade/scratch/mcamron/archive/f.c54120.FWscHIST.f09_f09.gwdst/atm/hist/f.c54120.FWscHIST.f09_f09.gwdst.cam.h1.2002-07-*.nc')
ds = ds.sel(lat=slice(-20,20))
ds

<xarray.Dataset>
Dimensions:          (ilev: 71, lat: 42, lev: 70, lon: 288, nbnd: 2, time: 744)
Coordinates:
  * lat              (lat) float64 -19.32 -18.38 -17.43 -16.49 -15.55 -14.61 ...
  * lon              (lon) float64 0.0 1.25 2.5 3.75 5.0 6.25 7.5 8.75 10.0 ...
  * lev              (lev) float64 5.96e-06 9.827e-06 1.62e-05 2.671e-05 ...
  * ilev             (ilev) float64 4.5e-06 7.42e-06 1.223e-05 2.017e-05 ...
  * time             (time) datetime64[ns] 2002-07-01 2002-07-01T00:59:59 ...
Dimensions without coordinates: nbnd
Data variables:
    gw               (time, lat) float64 0.01552 0.01561 0.01569 0.01577 ...
    hyam             (time, lev) float64 5.96e-09 9.827e-09 1.62e-08 ...
    hybm             (time, lev) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    P0               (time) float64 1e+05 1e+05 1e+05 1e+05 1e+05 1e+05 ...
    hyai             (time, ilev) float64 4.5e-09 7.42e-09 1.223e-08 ...
    hybi             (time, ilev) float64 0.0 0.0 0.0 0.0 0.0 0.

In [ ]:
pconvt = -7000 * np.log(ds['PCONVT'] / ds['PS'])    # Converting from pressure to height estimate

In [ ]:
f, ax = plt.subplots(figsize=(7,6))
pconvt.where(pconvt > 0).plot.hist(ax=ax, log=True, histtype='step', linewidth=2, label='PCONVT')

ax.legend()
ax.set_title('Distribution of Nonzero Cloud Height Fields, 20' + u'\N{DEGREE SIGN}' + 'S -- 20' + u'\N{DEGREE SIGN}' + 'N, July 2002')
ax.set_xlabel('Cloud Top Height [m]')

plt.tight_layout()    
f.savefig('../figures/clouds/hght_dist.png')

### Investigating CLOUDFRAC_CLUBB
First, looking at distribution of cloud fractions

In [ ]:
(ds['CLOUDFRAC_CLUBB'].where(ds['CLOUDFRAC_CLUBB'] > 0)).plot.hist(histtype='step')

Looking at heights of all nonzero CLUBB cloudfractions

In [ ]:
(xu.log(ds['lev'] / 1000) * -7000).where(ds['CLOUDFRAC_CLUBB'] > 0).plot.hist(log=True, histtype='step')

Does `CLOUDFRAC_CLUBB` actually include ZM clouds? Investigating `club_intr.F90` shows not so, but `CLOUDCOVER_CLUBB` may.

#### Finding max `lev` indices of `CLOUDFRAC_CLUBB > 0`

In [4]:
ccstack = ds['CLOUDFRAC_CLUBB'].stack(xyt=('time', 'lat', 'lon'))
indmax = ccstack.values.nonzero()
ccstack

In [10]:
ccstack

<xarray.DataArray 'CLOUDFRAC_CLUBB' (lev: 70, xyt: 8999424)>
dask.array<reshape, shape=(70, 8999424), dtype=float32, chunksize=(70, 290304)>
Coordinates:
  * lev      (lev) float64 5.96e-06 9.827e-06 1.62e-05 2.671e-05 4.404e-05 ...
  * xyt      (xyt) MultiIndex
  - time     (xyt) datetime64[ns] 2002-07-01 2002-07-01 2002-07-01 ...
  - lat      (xyt) float64 -19.32 -19.32 -19.32 -19.32 -19.32 -19.32 -19.32 ...
  - lon      (xyt) float64 0.0 1.25 2.5 3.75 5.0 6.25 7.5 8.75 10.0 11.25 ...
Attributes:
    mdims:      1
    units:      fraction
    long_name:  Cloud Fraction

In [24]:
indmax = np.array(indmax)
indmax

array([[     44,      44,      44, ...,      69,      69,      69],
       [3397955, 6059881, 6071977, ..., 8999247, 8999272, 8999364]])

In [5]:
for xyt in indmax[1]: print(ccstack.sel(xyt=xyt))

<xarray.DataArray 'CLOUDFRAC_CLUBB' (lev: 70)>
dask.array<getitem, shape=(70,), dtype=float32, chunksize=(70,)>
Coordinates:
  * lev      (lev) float64 5.96e-06 9.827e-06 1.62e-05 2.671e-05 4.404e-05 ...
    xyt      object (Timestamp('2002-07-01 00:00:00'), -19.319371727748688, 0.0)
Attributes:
    mdims:      1
    units:      fraction
    long_name:  Cloud Fraction
<xarray.DataArray 'CLOUDFRAC_CLUBB' (lev: 70)>
dask.array<getitem, shape=(70,), dtype=float32, chunksize=(70,)>
Coordinates:
  * lev      (lev) float64 5.96e-06 9.827e-06 1.62e-05 2.671e-05 4.404e-05 ...
    xyt      object (Timestamp('2002-07-01 00:00:00'), -19.319371727748688, 1.25) ...
Attributes:
    mdims:      1
    units:      fraction
    long_name:  Cloud Fraction
<xarray.DataArray 'CLOUDFRAC_CLUBB' (lev: 70)>
dask.array<getitem, shape=(70,), dtype=float32, chunksize=(70,)>
Coordinates:
  * lev      (lev) float64 5.96e-06 9.827e-06 1.62e-05 2.671e-05 4.404e-05 ...
    xyt      object (Timestamp('2002-07-01 00:00: